In [17]:
import hazelcast

# Konfiguracja klienta Hazelcast
client = hazelcast.HazelcastClient(
            cluster_name = "dev",
            cluster_members=["127.0.0.1:5701", "127.0.0.1:5702", "127.0.0.1:5703"],
        )


In [ ]:
from hazelcast.aggregator import count, number_avg, max_by
from hazelcast.predicate import less_or_equal

imap = client.get_map("coinbase_trades").blocking()
  
# # Pobranie danych z IMap
btc_usd_data = imap.get()
print("BTC-USD Data from Hazelcast:", btc_usd_data)



BTC-USD Data from Hazelcast: None


In [19]:
# Get and fill a map with some integers
integers = client.get_map("integers").blocking()
for i in range(100):
    integers.set(i, i)

# Create mapping for the integers. This needs to be done only once per map.
client.sql.execute(
    """
CREATE MAPPING integers
TYPE IMap
OPTIONS (
  'keyFormat' = 'int',
  'valueFormat' = 'int'
)
    """
).result()

# Fetch values in between (40, 50)
result_future = client.sql.execute("SELECT * FROM integers WHERE this > ? AND this < ?", 40, 50)


HazelcastSqlError: The Jet engine is disabled.
To enable the Jet engine on the members, do one of the following:
  - Change member config using Java API: config.getJetConfig().setEnabled(true)
  - Change XML/YAML configuration property: Set hazelcast.jet.enabled to true
  - Add system property: -Dhz.jet.enabled=true (for Hazelcast embedded, works only when loading config via Config.load)
  - Add environment variable: HZ_JET_ENABLED=true (recommended when running container image. For Hazelcast embedded, works only when loading config via Config.load)

In [ ]:
# Zamknięcie klienta Hazelcast
client.shutdown()

#### Client Jet
Jet Streams to mechanizm umożliwiający obsługę strumieni danych w czasie rzeczywistym. Pozwala on na przetwarzanie strumieni danych (np. z WebSocketów) i ich przesyłanie do innych komponentów systemu, takich jak IMap w Hazelcast.

In [7]:
import hazelcast
from hazelcast.jet import JetClient

# Tworzymy klienta Hazelcast Jet
client = hazelcast.HazelcastClient(
    cluster_name="dev",  # Nazwa klastra
    network_config={
        "tcp_ip": {
            "enabled": True,
            "member_list": ["127.0.0.1:5701", "127.0.0.1:5702", "127.0.0.1:5703"]
        },
    },
)

# Tworzymy klienta Jet
jet_client = JetClient.new_client()

# Uzyskujemy dostęp do mapy
map_name = "trades"
imap = client.get_map(map_name).blocking()

# Sprawdzamy połączenie
print(f"Connected to Hazelcast Jet cluster {client.get_cluster().get_cluster_id()}")


ModuleNotFoundError: No module named 'hazelcast.jet'